In [14]:
from tqdm import tqdm
from CellClass import Analyse as an
import numpy as np

import os

base = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/MYCN_SpikeIn/results/patches"
negative = [os.path.join(base, x) for x in os.listdir(base) if "S29" in x]
positive = [os.path.join(base, x) for x in os.listdir(base) if "S19" in x]

negatives = []
for f in tqdm(negative):
    negatives.extend(an.load_patches(f))
    
positives = []
for f in tqdm(positive):
    positives.extend(an.load_patches(f))

100%|██████████| 88/88 [01:04<00:00,  1.36it/s]


In [15]:
r = min(int(len(positives)/100), int(len(negatives)/100))
print(len(negatives), len(positives))
positives = positives[:r*100]
negatives = negatives[:r*100]
len(negatives), len(positives)

9910 16179


(9900, 9900)

In [16]:
train_p = np.array([[y for y in x.Features] for x in positives])
target_p = np.ones(train_p.shape[0])

train_n = np.array([[y for y in x.Features] for x in negatives])
target_n = np.zeros(target_p.shape[0])

In [17]:
train = np.concatenate((train_p, train_n))
target = np.concatenate((target_p, target_n))
train.shape, target.shape

((19800, 21), (19800,))

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.25, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14850, 21), (4950, 21), (14850,), (4950,))

In [19]:
X_train[0], X_test[0], y_train[0], y_test[0]

(array([0.27678573, 0.3482143 , 0.39285716, 0.46875003, 0.51785719,
        0.59375   , 0.80803573, 0.4810127 , 0.62025321, 0.75105488,
        0.87974685, 1.        , 1.        , 1.        , 0.36554623,
        0.46638659, 0.56302524, 0.65546221, 0.73949581, 0.78151262,
        0.8487395 ]),
 array([0.06486487, 0.07027028, 0.07567568, 0.08108108, 0.08648649,
        0.09189189, 0.15675676, 0.05416667, 0.05833333, 0.06666666,
        0.08333334, 0.10833334, 0.17500001, 0.42416668, 0.25941426,
        0.29288703, 0.33472803, 0.38912135, 0.42259416, 0.44351465,
        0.47698748]),
 1.0,
 1.0)

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [21]:
predictions = model.predict(X_test)

In [22]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predictions)

TN, FP, FN, TP = confusion_matrix(y_test, predictions).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

accuracy =  (TP+TN) /(TP+FP+TN+FN)

print('Accuracy of the binary classification = {:0.3f}'.format(accuracy))

True Positive(TP)  =  2284
False Positive(FP) =  61
True Negative(TN)  =  2426
False Negative(FN) =  179
Accuracy of the binary classification = 0.952
